#### Import libraries:

In [2]:
import os
import cv2
import numpy as np


# Data Augmentation Libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
import random


from sklearn.model_selection import train_test_split

#from sklearn.metrics import accuracy_score
#from sklearn.metrics import precision_score
#from sklearn.metrics import recall_score
#from sklearn.metrics import f1_score

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
#from tensorflow.keras.datasets import mnist
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Conv2D
#from tensorflow.keras.layers import MaxPool2D
#from tensorflow.keras.layers import Flatten
#from tensorflow.keras.layers import Dropout
#from tensorflow.keras.layers import Dense



import matplotlib.pyplot as plt

#import pickle

#import matplotlib.pyplot as plt

In [3]:
import warnings
warnings.filterwarnings("ignore")

#### Workflow:
- Crear entorno nuevo con versión python 3.11
- instalar tensorflow
- Data Augmentation
- aplicar una CNN
- reserch de los parámetros y de la estructura: feed forward? capas? dense layers? loss functions? auc? batch? epochs?

##### Data Augmentation Functions

In [ ]:
# Rotation, Noise, Flipping

In [ ]:
def add_noise(img):
    '''Add random noise to an image'''
    VARIABILITY = 50
    deviation = VARIABILITY*random.random()
    noise = np.random.normal(0, deviation, img.shape)
    new_img = img + noise
    np.clip(new_img, 0., 255.)
    return new_img

'''
def change_contrast(img):      
    for i in range(3):
      seed = (i, 0)
      contrast_image = tf.image.stateless_random_contrast(
          img, lower=0.1, upper=0.9, seed=seed)
      return contrast_image
'''

# Initialising the ImageDataGenerator class.
# We will pass in the augmentation parameters in the constructor.
datagen = ImageDataGenerator(
        rotation_range = 360,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip = True,
        vertical_flip = True
        preprocessing_function=add_noise,


def augmentation(image_path, new_image_paths):
    # Loading a sample image 
    img = load_img(image_path) 
    # Converting the input sample image to an array
    x = img_to_array(img)
    # Reshaping the input image
    x = x.reshape((1, ) + x.shape)     

    # Generating and saving 5 augmented samples 
    # using the above defined parameters. 
    i = 0
    for batch in datagen.flow(x, batch_size = 5,
                              save_to_dir ='', 
                              save_prefix ='', save_format ='jpeg'): # mette automaticamente .1, .2 come suffisso
        i+=1
        if i==4:
            break

##### CNN

In [7]:
# Define your CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
# Add more layers as needed
model.add(layers.Flatten())
model.add(layers.Dense(100, activation='softmax'))  # 100 output classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Create an ImageDataGenerator for data augmentation
def add_noise(img):
    #Add random noise to an image
    VARIABILITY = 50
    deviation = VARIABILITY*random.random()
    noise = np.random.normal(0, deviation, img.shape)
    new_img = img + noise
    np.clip(new_img, 0., 255.)
    return new_img
    
# Initialising the ImageDataGenerator class.
# We will pass in the augmentation parameters in the constructor.
datagen = ImageDataGenerator(
        rotation_range = 360,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip = True,
        vertical_flip = True
        preprocessing_function=add_noise
)

In [ ]:
# Load your dataset and split it into train, validation, and test sets

# LOADING

base_dir = '../data/dataset'
all_image_paths = []
all_labels = []

# Walk through the directory structure
for label in os.listdir(base_dir):
    label_path = os.path.join(base_dir, label)

    # Walk through the images in the subdirectory
    for image_filename in os.listdir(label_path):
        image_path = os.path.join(label_path, image_filename)

        # Add the image path and corresponding label to the lists
        all_image_paths.append(image_path)
        all_labels.append(label)

# Ensure that all_image_paths and all_labels are numpy arrays for easier manipulation
image_paths = np.array(all_image_paths)
labels = np.array(all_labels)


# SPLITTING

# Split the dataset into training and temporary sets (combined validation and test)
train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    image_paths, labels, test_size=0.3, random_state=42
)

# Split the temporary set into validation and test sets
val_paths, test_paths, val_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=0.5, random_state=42
)

In [ ]:
# Train the model with data augmentation
batch_size = 32 # number of training examples utilized in one iteration

model.fit_generator(
    datagen.flow(train_data, train_labels, batch_size=batch_size),
    steps_per_epoch = len(train_data) / batch_size,  # adjust batch_size and steps_per_epoch accordingly
    epochs=50,
    validation_data=(val_data, val_labels)
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_data, test_labels)
print(f'Test accuracy: {test_acc}')


In the line datagen.flow(train_data, train_labels, batch_size=batch_size), you are using the flow method of the ImageDataGenerator to create a generator that yields augmented batches of training data on-the-fly.

Here's a breakdown of each parameter:

train_data: This is the input training data, typically a numpy array or a directory path. In the context of ImageDataGenerator, it is expected to be a 4D array representing the images (samples) in the training dataset.

train_labels: This is the corresponding labels for the training data. It could be one-hot encoded or categorical labels.

batch_size: This parameter specifies the number of samples in each batch that the generator will yield.

When you use datagen.flow(train_data, train_labels, batch_size=batch_size), the ImageDataGenerator takes care of applying the specified data augmentation techniques (e.g., rotation, width shift, height shift, horizontal flip, vertical flip, and your custom noise function add_noise) to the training data on-the-fly. It generates batches of augmented data during each iteration of training.

Here's a simplified explanation of how the flow works:

The generator takes a batch of batch_size samples from the training data.
It applies the specified data augmentation techniques to each sample in the batch.
It yields the augmented batch to the model for training.
This process is repeated for each epoch during training. It allows you to artificially increase the size of your training dataset and introduce variability to improve the model's ability to generalize to unseen data.

In [ ]:
#loading data
(X_train,y_train) , (X_test,y_test)=mnist.load_data()

#reshaping data
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
X_test = X_test.reshape((X_test.shape[0],X_test.shape[1],X_test.shape[2],1))

#checking the shape after reshaping
print(X_train.shape)
print(X_test.shape)

#normalizing the pixel values
X_train=X_train/255
X_test=X_test/255

#defining model
model=Sequential()

#adding convolution layer
model.add(Conv2D(32,(3,3),activation=’relu’,input_shape=(28,28,1)))

#adding pooling layer
model.add(MaxPool2D(2,2))

#adding fully connected layer
model.add(Flatten())
model.add(Dense(100,activation=’relu’))

#adding output layer
model.add(Dense(10,activation=’softmax’))

#compiling the model
model.compile(loss=’sparse_categorical_crossentropy’,optimizer=’adam’,metrics=[‘accuracy’])

#fitting the model
model.fit(X_train,y_train,epochs=10)

#evaluting the model
model.evaluate(X_test,y_test)

#### Functions:

In [ ]:
def load_images_from_folder(folder):
    # Function to load and preprocess images from a folder
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
            img = cv2.resize(img, (64, 64))  # Resize image ###########################
            images.append(img)
    return images

In [ ]:
def get_folders_in_directory(directory_path):
    # Get the list of all files and folders in the specified directory
    items = os.listdir(directory_path)

    # Filter out only the folders from the list
    folders = [item for item in items if os.path.isdir(os.path.join(directory_path, item))]

    return folders

#### 'main':

In [ ]:
# Specify the path of the directory you want to list folders from
directory_path = '../data/dataset'

# Call the function to get the list of folders
asana = get_folders_in_directory(directory_path)

In [ ]:
# Load images and give labels
X=[]
y=[]
for asana_name in asana:
    path = directory_path + asana_name
    X = X + load_images_from_folder(asana_name)
    y.append(asana_name)

In [ ]:
# Convert data to NumPy arrays
X = np.array(X)
y = np.array(y)

In [ ]:
# Flatten the image data
X = X.reshape(X.shape[0], -1) # ??????????????????

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40) 
'''
random_state : int, default=None. Controls the shuffling applied to the data before applying the split.
                Pass an int for reproducible output across multiple function calls.
'''

In [ ]:
# Do the following for different models and compare


# Train logistic regression model
'''
ex:
model = ()
model.fit(X_train, y_train)
'''

# Evaluate model on test set
'''
ex:
y_pred = model.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1 score: {f1_score(y_test, y_pred)}")
'''


In [ ]:
# Export model

'''
model_file = '../models/asana_model.pkl'
with open(model_file, 'wb') as f:
    pickle.dump(model, f)
'''

In [ ]:
# Import model
model_file = '../models/logistic_cats_dogs.pkl' # pickle (pkl), .h5, ...
with open(model_file, 'rb') as f:
    model = pickle.load(f)